#### **Install Libraries**

In [1]:
!pip install torch
!pip install torch-geometric
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-<YOUR-TORCH-VERSION>+cpu.html
!pip install torch torchvision torchaudio torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 18.6 MB/s eta 0:00:00
/bin/bash: line 1: YOUR-TORCH-VERSION: No such file or directory


#### **Import required Libraries**

In [2]:
import torch                                      # The main PyTorch library for tensor computation.
import torch.nn as nn                             # Provides classes and functions for building neural networks.
import torch.optim as optim                       # Contains various optimization algorithms for training neural networks.
from torch_geometric.nn import GCNConv            # A Graph Convolutional Layer from PyTorch Geometric.
from torch_geometric.nn import GATConv            # A Graph Attention Network (GAT) Convolutional Layer from PyTorch Geometric.
from torch_geometric.nn import SAGEConv           # A GraphSAGE Convolutional Layer from PyTorch Geometric.
from torch_geometric.nn import TransformerConv    # A Transformer Convolutional Layer from PyTorch Geometric.
import torch.nn.functional as F                   # Contains various functions for building neural networks (e.g., activation functions).
from torch_geometric.data import Data             # A class for graph data in PyTorch Geometric.
from torch.optim import Adam                      # Adam optimization algorithm for training neural networks.
from torch.nn.functional import cross_entropy     # Cross-entropy loss function for classification tasks.
from torch_geometric.loader import NeighborLoader # Loads graph data with neighbor sampling for efficient training.
from torch_geometric.loader import DataLoader     # A DataLoader for graph data in PyTorch Geometric.
import networkx as nx                             # Library for graph and network analysis.
from torch_geometric.utils import from_networkx   # Converts a NetworkX graph to PyTorch Geometric format.
from sklearn.preprocessing import StandardScaler  # Standardizes features by removing the mean and scaling to unit variance.
from sklearn.model_selection import train_test_split # Splits datasets into training and testing subsets.
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score # Metrics for evaluating models.
from tqdm import tqdm                             # For progress tracking in loops.
import networkx as nx                             # Library for creating and analyzing graph data structures.
import pickle                                     # For saving and loading Python objects (e.g., models, data).
import os                                         # For file and directory operations.
import itertools                                  # For working with iterators and combinations.
from itertools import product                     # For generating the Cartesian product of input iterables.
# Import python packages
import pandas as pd                               # For data manipulation and analysis.
import numpy as np                                # For numerical computations.
import seaborn as sns                             # For data visualization.
import matplotlib.pyplot as plt                   # For creating visualizations.
# To ignore warnings
import warnings                                   # Handles Python warnings.
warnings.filterwarnings("ignore")                # Suppresses all warnings.


#### **Load the Stratified Heterogeneous Graph**

In [3]:
# Load the stratified heterogeneous graph
graph_path = '/content/drive/MyDrive/GraphFeatures/StratifiedHeteroGraph/StratifiedHeteronousGraph_AllEdges.pt'
hetero_data = torch.load(graph_path)

print(f"Graph Loaded: {hetero_data}")
print(f"Node features shape: {hetero_data.x.size()}")
print(f"Edge index shape: {hetero_data.edge_index.size()}")
print(f"Target labels shape: {hetero_data.y.size()}")

Graph Loaded: Data(x=[99898, 101], edge_index=[2, 23504013], y=[99898])
Node features shape: torch.Size([99898, 101])
Edge index shape: torch.Size([2, 23504013])
Target labels shape: torch.Size([99898])


#### **Scale Node Features**

In [4]:
# Scale node features using StandardScaler
scaler = StandardScaler()
hetero_data.x = torch.tensor(
    scaler.fit_transform(hetero_data.x.cpu().numpy()),
    dtype=torch.float
).to(hetero_data.x.device)  # Ensure the scaled features are on the correct device
print("Node features scaled successfully.")

Node features scaled successfully.


#### **Train-Val-Test Split**

In [5]:
# Split the nodes into train, validation, and test sets
num_nodes = hetero_data.num_nodes
train_size = int(0.6 * num_nodes)
val_size = int(0.2 * num_nodes)
test_size = num_nodes - train_size - val_size

# Generate random permutations for shuffling the nodes
perm = torch.randperm(num_nodes)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[perm[:train_size]] = True
val_mask[perm[train_size:train_size + val_size]] = True
test_mask[perm[train_size + val_size:]] = True

# Assign the masks to the graph
hetero_data.train_mask = train_mask
hetero_data.val_mask = val_mask
hetero_data.test_mask = test_mask

print("Train, validation, and test masks created successfully.")

Train, validation, and test masks created successfully.


#### **Stratified Heterogenous Graph with GCN model**

#### **Define Model**

In [6]:
class GCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

##### **Initialize and Train the Model**

In [7]:
# Model parameters
input_dim = hetero_data.x.size(1)  # Feature dimension
hidden_dim = 64
output_dim = len(torch.unique(hetero_data.y))  # Number of classes

model = GCNModel(input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Move data and model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
hetero_data = hetero_data.to(device)

# Training loop
num_epochs = 50
best_val_loss = float('inf')
patience = 5
stopping_counter = 0

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    out = model(hetero_data.x, hetero_data.edge_index)

    # Compute loss on training nodes
    loss = criterion(out[hetero_data.train_mask], hetero_data.y[hetero_data.train_mask])
    loss.backward()
    optimizer.step()

    # Validation loss
    model.eval()
    with torch.no_grad():
        val_out = model(hetero_data.x, hetero_data.edge_index)
        val_loss = criterion(val_out[hetero_data.val_mask], hetero_data.y[hetero_data.val_mask])

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        stopping_counter = 0
    else:
        stopping_counter += 1
        if stopping_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


Epoch 1/50, Training Loss: 0.6965, Validation Loss: 0.7686
Epoch 2/50, Training Loss: 0.7691, Validation Loss: 0.7079
Epoch 3/50, Training Loss: 0.7084, Validation Loss: 0.6500
Epoch 4/50, Training Loss: 0.6503, Validation Loss: 0.6684
Epoch 5/50, Training Loss: 0.6689, Validation Loss: 0.6894
Epoch 6/50, Training Loss: 0.6902, Validation Loss: 0.6761
Epoch 7/50, Training Loss: 0.6772, Validation Loss: 0.6534
Epoch 8/50, Training Loss: 0.6545, Validation Loss: 0.6454
Epoch 9/50, Training Loss: 0.6465, Validation Loss: 0.6525
Epoch 10/50, Training Loss: 0.6536, Validation Loss: 0.6615
Epoch 11/50, Training Loss: 0.6625, Validation Loss: 0.6626
Epoch 12/50, Training Loss: 0.6635, Validation Loss: 0.6563
Epoch 13/50, Training Loss: 0.6570, Validation Loss: 0.6488
Early stopping at epoch 13


#### **Evaluate the Model**

In [8]:
# Evaluate the model
model.eval()
with torch.no_grad():
    out = model(hetero_data.x, hetero_data.edge_index)
    preds = out.argmax(dim=1).cpu().numpy()  # Predicted labels

    # Test set evaluation
    y_true = hetero_data.y[hetero_data.test_mask].cpu().numpy()
    y_pred = preds[hetero_data.test_mask.cpu().numpy()]

    # Metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, out[hetero_data.test_mask][:, 1].cpu().numpy())

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")


Accuracy: 0.6451
Precision: 0.6741
Recall: 0.5607
F1-score: 0.6122
AUC-ROC: 0.6515


#### **Stratified Heterogenous Graph with GAT Model**

#### **Define Model**

In [9]:
class GATModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=1):
        super(GATModel, self).__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads, concat=True)
        self.gat2 = GATConv(hidden_dim * heads, output_dim, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = self.gat2(x, edge_index)
        return x

#### **Initialize and Train Model**

In [10]:
# Model parameters
input_dim = hetero_data.x.size(1)  # Feature dimension
hidden_dim = 64
output_dim = len(torch.unique(hetero_data.y))  # Number of classes
heads = 8  # Number of attention heads

model = GATModel(input_dim, hidden_dim, output_dim, heads=heads)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Move data and model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
hetero_data = hetero_data.to(device)

# Training loop
num_epochs = 50
best_val_loss = float('inf')
patience = 5
stopping_counter = 0

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    out = model(hetero_data.x, hetero_data.edge_index)

    # Compute loss on training nodes
    loss = criterion(out[hetero_data.train_mask], hetero_data.y[hetero_data.train_mask])
    loss.backward()
    optimizer.step()

    # Validation loss
    model.eval()
    with torch.no_grad():
        val_out = model(hetero_data.x, hetero_data.edge_index)
        val_loss = criterion(val_out[hetero_data.val_mask], hetero_data.y[hetero_data.val_mask])

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        stopping_counter = 0
    else:
        stopping_counter += 1
        if stopping_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


Epoch 1/50, Training Loss: 1.0115, Validation Loss: 1.8357
Epoch 2/50, Training Loss: 1.8339, Validation Loss: 0.9291
Epoch 3/50, Training Loss: 0.9310, Validation Loss: 1.2624
Epoch 4/50, Training Loss: 1.2597, Validation Loss: 0.8245
Epoch 5/50, Training Loss: 0.8254, Validation Loss: 0.8966
Epoch 6/50, Training Loss: 0.8993, Validation Loss: 0.7432
Epoch 7/50, Training Loss: 0.7466, Validation Loss: 0.7689
Epoch 8/50, Training Loss: 0.7744, Validation Loss: 0.7017
Epoch 9/50, Training Loss: 0.7025, Validation Loss: 0.6968
Epoch 10/50, Training Loss: 0.6948, Validation Loss: 0.6851
Epoch 11/50, Training Loss: 0.6820, Validation Loss: 0.7081
Epoch 12/50, Training Loss: 0.7059, Validation Loss: 0.6759
Epoch 13/50, Training Loss: 0.6744, Validation Loss: 0.6814
Epoch 14/50, Training Loss: 0.6829, Validation Loss: 0.6727
Epoch 15/50, Training Loss: 0.6754, Validation Loss: 0.6622
Epoch 16/50, Training Loss: 0.6638, Validation Loss: 0.6743
Epoch 17/50, Training Loss: 0.6748, Validation Lo

#### **Evaluate Model**

In [11]:
# Evaluate the model
model.eval()
with torch.no_grad():
    out = model(hetero_data.x, hetero_data.edge_index)
    preds = out.argmax(dim=1).cpu().numpy()  # Predicted labels

    # Test set evaluation
    y_true = hetero_data.y[hetero_data.test_mask].cpu().numpy()
    y_pred = preds[hetero_data.test_mask.cpu().numpy()]

    # Metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, out[hetero_data.test_mask][:, 1].cpu().numpy())

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")

Accuracy: 0.6448
Precision: 0.6837
Recall: 0.5376
F1-score: 0.6019
AUC-ROC: 0.5969


#### **Stratified Heterogenous Graph with GraphSAGE Model**

#### **Define Model**

In [12]:
class GraphSAGEModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphSAGEModel, self).__init__()
        self.sage1 = SAGEConv(input_dim, hidden_dim)
        self.sage2 = SAGEConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.sage1(x, edge_index)
        x = F.relu(x)
        x = self.sage2(x, edge_index)
        return x


#### **Initialize and Train Model**

In [13]:
# Model parameters
input_dim = hetero_data.x.size(1)  # Feature dimension
hidden_dim = 64
output_dim = len(torch.unique(hetero_data.y))  # Number of classes

model = GraphSAGEModel(input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Move data and model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
hetero_data = hetero_data.to(device)

# Training loop
num_epochs = 50
best_val_loss = float('inf')
patience = 5
stopping_counter = 0

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    out = model(hetero_data.x, hetero_data.edge_index)

    # Compute loss on training nodes
    loss = criterion(out[hetero_data.train_mask], hetero_data.y[hetero_data.train_mask])
    loss.backward()
    optimizer.step()

    # Validation loss
    model.eval()
    with torch.no_grad():
        val_out = model(hetero_data.x, hetero_data.edge_index)
        val_loss = criterion(val_out[hetero_data.val_mask], hetero_data.y[hetero_data.val_mask])

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        stopping_counter = 0
    else:
        stopping_counter += 1
        if stopping_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


Epoch 1/50, Training Loss: 0.7223, Validation Loss: 0.6191
Epoch 2/50, Training Loss: 0.6180, Validation Loss: 0.5980
Epoch 3/50, Training Loss: 0.5957, Validation Loss: 0.5591
Epoch 4/50, Training Loss: 0.5560, Validation Loss: 0.5249
Epoch 5/50, Training Loss: 0.5213, Validation Loss: 0.5069
Epoch 6/50, Training Loss: 0.5033, Validation Loss: 0.4939
Epoch 7/50, Training Loss: 0.4906, Validation Loss: 0.4777
Epoch 8/50, Training Loss: 0.4747, Validation Loss: 0.4605
Epoch 9/50, Training Loss: 0.4576, Validation Loss: 0.4465
Epoch 10/50, Training Loss: 0.4433, Validation Loss: 0.4346
Epoch 11/50, Training Loss: 0.4308, Validation Loss: 0.4213
Epoch 12/50, Training Loss: 0.4168, Validation Loss: 0.4058
Epoch 13/50, Training Loss: 0.4007, Validation Loss: 0.3896
Epoch 14/50, Training Loss: 0.3842, Validation Loss: 0.3745
Epoch 15/50, Training Loss: 0.3693, Validation Loss: 0.3598
Epoch 16/50, Training Loss: 0.3554, Validation Loss: 0.3446
Epoch 17/50, Training Loss: 0.3405, Validation Lo

#### **Evaluate Model**

In [14]:
# Evaluate the model
model.eval()
with torch.no_grad():
    out = model(hetero_data.x, hetero_data.edge_index)
    preds = out.argmax(dim=1).cpu().numpy()  # Predicted labels

    # Test set evaluation
    y_true = hetero_data.y[hetero_data.test_mask].cpu().numpy()
    y_pred = preds[hetero_data.test_mask.cpu().numpy()]

    # Metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, out[hetero_data.test_mask][:, 1].cpu().numpy())

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")

Accuracy: 0.9991
Precision: 0.9986
Recall: 0.9997
F1-score: 0.9991
AUC-ROC: 0.9999


#### **Stratified Heterogenous Graph with Graphomer Transformer Model**

##### **Define Model**

In [15]:
class GraphomerModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads=4):
        super(GraphomerModel, self).__init__()
        self.transformer1 = TransformerConv(input_dim, hidden_dim, heads=num_heads)
        self.transformer2 = TransformerConv(hidden_dim * num_heads, hidden_dim, heads=num_heads)
        self.fc = nn.Linear(hidden_dim * num_heads, output_dim)

    def forward(self, x, edge_index):
        x = self.transformer1(x, edge_index)
        x = F.relu(x)
        x = self.transformer2(x, edge_index)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.fc(x)
        return x

##### **Initialize and Train Model**

In [16]:
# Model parameters
input_dim = hetero_data.x.size(1)  # Feature dimension
hidden_dim = 32
output_dim = len(torch.unique(hetero_data.y))  # Number of classes
num_heads = 4

model = GraphomerModel(input_dim, hidden_dim, output_dim, num_heads)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Early stopping parameters
patience = 5
best_val_loss = float('inf')
stopping_counter = 0

# Move data and model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
hetero_data = hetero_data.to(device)

# Training Loop
num_epochs = 50
patience = 5
best_val_loss = float('inf')
stopping_counter = 0
early_stop = False

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    out = model(hetero_data.x, hetero_data.edge_index)
    loss = criterion(out[hetero_data.train_mask], hetero_data.y[hetero_data.train_mask])
    loss.backward()
    optimizer.step()

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_out = model(hetero_data.x, hetero_data.edge_index)
        val_loss = criterion(val_out[hetero_data.val_mask], hetero_data.y[hetero_data.val_mask])

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        stopping_counter = 0
    else:
        stopping_counter += 1
        if stopping_counter >= patience:
            early_stop = True

Epoch 1/50, Training Loss: 0.7485, Validation Loss: 1.1063
Epoch 2/50, Training Loss: 1.1138, Validation Loss: 0.6731
Epoch 3/50, Training Loss: 0.6743, Validation Loss: 0.6115
Epoch 4/50, Training Loss: 0.6102, Validation Loss: 0.5757
Epoch 5/50, Training Loss: 0.5751, Validation Loss: 0.5704
Epoch 6/50, Training Loss: 0.5717, Validation Loss: 0.5205
Epoch 7/50, Training Loss: 0.5223, Validation Loss: 0.4696
Epoch 8/50, Training Loss: 0.4696, Validation Loss: 0.4539
Epoch 9/50, Training Loss: 0.4530, Validation Loss: 0.4352
Epoch 10/50, Training Loss: 0.4346, Validation Loss: 0.3934
Epoch 11/50, Training Loss: 0.3923, Validation Loss: 0.3551
Epoch 12/50, Training Loss: 0.3538, Validation Loss: 0.3297
Epoch 13/50, Training Loss: 0.3252, Validation Loss: 0.2888
Epoch 14/50, Training Loss: 0.2805, Validation Loss: 0.2566
Epoch 15/50, Training Loss: 0.2460, Validation Loss: 0.2067
Epoch 16/50, Training Loss: 0.1949, Validation Loss: 0.1667
Epoch 17/50, Training Loss: 0.1566, Validation Lo

##### **Evaluate Model**

In [17]:
# Evaluate the model
model.eval()
with torch.no_grad():
    out = model(hetero_data.x, hetero_data.edge_index)
    preds = out.argmax(dim=1).cpu().numpy()  # Predicted labels

    # Test set evaluation
    y_true = hetero_data.y[hetero_data.test_mask].cpu().numpy()
    y_pred = preds[hetero_data.test_mask.cpu().numpy()]

    # Metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, out[hetero_data.test_mask][:, 1].cpu().numpy())

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")

Accuracy: 0.9994
Precision: 0.9996
Recall: 0.9992
F1-score: 0.9994
AUC-ROC: 1.0000
